In [ ]:
pip install lightfm

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

DIR = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/recommendationsv4/'
DIR_TEST = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/TEST/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
#%matplotlib inline
import numpy as np # linear algebra
import pandas as pd



In [ ]:
#Читаем файлы
data = pd.read_csv(DIR + 'train.csv',low_memory=False )
test = pd.read_csv(DIR +'test.csv',low_memory=False )
submission = pd.read_csv(DIR +'sample_submission.csv')

In [ ]:
set_asin = set(data['asin']) | set(test['asin'])
len(set_asin)

41320

In [ ]:
# Проверим , что между  asin и itemid однозначное соответствие, что данные корректны
data_all = pd.concat([data, test])
data_all_test = pd.DataFrame(data_all.groupby(['asin']).itemid.max() - data_all.groupby(['asin']).itemid.min())
data_all_test.itemid.value_counts()
# должен быть ноль по всей длине датафрейма

0    41320
Name: itemid, dtype: int64

In [ ]:
#Словарь соответствия asin и itemid
mapper = dict(zip(data_all.asin, data_all.itemid))
len(mapper)

41320

In [ ]:
import json
def get_by_key(field, dict_from, dict_to):
   if field in dict_from.keys():
       dict_to[mapper.get(dict_from.get('asin'))] = dict_from.get(field)    

with open(DIR + 'meta_Grocery_and_Gourmet_Food_small.json') as file:
     s_tag = set() 
     d_main_category = {}
     d_brand = {}
     d_category = {}
     i = 0
     for line in file.readlines(): 
         lines = json.loads(line) 
         s_tag.update(lines.keys())
         get_by_key('brand', lines, d_brand)
         get_by_key('main_cat', lines, d_main_category)
         get_by_key('category', lines, d_category)

In [ ]:
# Множество всех itemid
set_itemid = set(data['itemid']) | set(test['itemid']) 

#Подготовим словари с фичами для модели, обеспечим уникальность значения
d_brand_ml =  {x:'brand:' + y for x,y in d_brand.items()}
d_main_category_ml = {x:'main_cat:' + y for x,y in d_main_category.items()}
#Поиграем самой категорией, возьмем самый нижний уровень
d_last_category_ml = {x:'last_cat:' + y[-1] for x,y in d_category.items()}

In [ ]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split



In [ ]:
#Формируем  mapping для всех пользователей и товаров(train + test)
from lightfm.data import Dataset

dataset = Dataset() #item_identity_features=False)
dataset.fit((x for x in pd.concat([data, test])['userid']),
            (x for x in pd.concat([data, test])['itemid']))

In [ ]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 127496, num_items 41320.


In [ ]:
# Добавляем фичи по товарам 
dataset.fit_partial(items=(x for x in d_last_category_ml.keys()),
                    item_features=(x for x in d_last_category_ml.values()))

dataset.fit_partial(items=(x for x in d_main_category_ml.keys()),
                    item_features=(x for x in d_main_category_ml.values()))

dataset.fit_partial(items=(x for x in d_brand_ml.keys()),
                    item_features=(x for x in d_brand_ml.values()))

In [ ]:
#Готовим основную матрицу
(interactions, weights) = dataset.build_interactions(((x,y,z) 
          for x,y, z in zip(data.userid.values,data.itemid.values, data.rating.values )))
         # for x,y, z in zip(train_data.userid.values,train_data.itemid.values, train_data.overall.values -3 )))
  
print(repr(interactions))

<127496x41320 sparse matrix of type '<class 'numpy.int32'>'
	with 857895 stored elements in COOrdinate format>


In [ ]:
#Разбиваем наш датасет на обучающую и тестовую выборки
train_data, test_data = random_train_test_split(interactions, test_percentage=0.25,
                                                       random_state=np.random.RandomState(32))
train_weights, test_weights = random_train_test_split(weights, test_percentage=0.25,
                                                       random_state=np.random.RandomState(32))

In [ ]:
# Готовим item_features
def get_item_features(itemid):
   def add_from_dict(dict_):
      if dict_.get(itemid):
         list_feat.append(dict_.get(itemid)) 

   list_feat =[]
   add_from_dict(d_brand_ml)
   add_from_dict(d_last_category_ml)
   add_from_dict(d_main_category_ml)
   return list_feat

item_features = dataset.build_item_features(((x, get_item_features(x))
                                               for x in set_itemid ))
print(repr(item_features))

<41320x51173 sparse matrix of type '<class 'numpy.float32'>'
	with 164772 stored elements in Compressed Sparse Row format>


In [ ]:
# Вытаскиваем словари для преобразований
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [ ]:
from lightfm import LightFM
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 120 #число параметров вектора 
NUM_EPOCHS = 15 #число эпох обучения


In [ ]:
from lightfm.evaluation import auc_score

model_warp = LightFM(learning_rate=0.1, loss='warp', no_components=NUM_COMPONENTS)
model_warp = model_warp.fit(train_data, sample_weight = train_weights, epochs=NUM_EPOCHS, num_threads=NUM_THREADS
                            ,item_features=item_features
                            ) 


In [ ]:
#Метрика из sklearn
preds_warp = model_warp.predict(test_data.row,test_data.col
                              ,item_features=item_features
                              )   

print(preds_warp.min(), preds_warp.max(),preds_warp.mean(),preds_warp.std())   

sklearn.metrics.roc_auc_score(test_weights.data,preds_warp)

-9.518194198608398 10.71720027923584 0.1959798640160907 1.9151263529881815


0.5741486545510677

In [ ]:
%%time
#Метрика из LightFM
test_auc = auc_score(model_warp,test_data, item_features=item_features,
                     num_threads=NUM_THREADS)

print(test_auc.mean())

0.81193775
CPU times: user 1h 9min 32s, sys: 1.14 s, total: 1h 9min 33s
Wall time: 35min 29s


In [ ]:
test_auc.mean()

0.81193775

In [ ]:
from lightfm.evaluation import precision_at_k, recall_at_k
prec_score = precision_at_k(
                     model_warp,
                     test_data,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=item_features
                     ).mean()
 
recall_at_k = recall_at_k(model_warp,
                     test_data,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=item_features).mean()

print(recall_at_k,prec_score)


0.11175120925212938 0.0206152


In [ ]:
#предсказание для kaggle 
preds = model_warp.predict(test.userid.apply(lambda x: user_id_map[x]).values,
                      test.itemid.apply(lambda x: item_id_map[x]).values
                     ,item_features=item_features 
                      )

Хорошее значение метрики auc_score из LightFM , но очень плохие значения для recall_at_k,prec_score, модель неудачная, очень слабый результат на каггле 0.58593